In [1]:
preferences = {
    "T3.1":  [4, 2, 5],
    "T3.2":  [2, 5, 3],
    "T3.3":  [3, 5, 4],
    "T3.4":  [2, 4, 3],
    "T3.5":  [3, 5, 4],
    "T3.6":  [5, 3, 4],
    "T3.7":  [5, 3, 5],
    "T3.8":  [4, 3, 5],
    "T3.9":  [5, 4, 5],
    "T3.10": [5, 4, 5],
    "T3.11": [4, 3, 5],
}

effort = {
    "T3.1": 6.0,
    "T3.2": 2.0,
    "T3.3": 3.2,
    "T3.4": 2.0,
    "T3.5": 3.0,
    "T3.6": 2.0,
    "T3.7": 2.0,
    "T3.8": 1.2,
    "T3.9": 3.0,
    "T3.10": 3.2,
    "T3.11": 4.0,
}


import pulp

tasks = list(preferences.keys())
participants = ["U1", "U2", "U3"]

MAX_LOAD = 14 

model = pulp.LpProblem("Phase2_OpenSolver_Equivalent", pulp.LpMaximize)

x = pulp.LpVariable.dicts(
    "assign",
    [(t, p) for t in tasks for p in participants],
    lowBound=0,
    upBound=1,
    cat="Integer"
)

model += pulp.lpSum(
    preferences[t][i] * x[(t, participants[i])]
    for t in tasks
    for i in range(3)
)


for t in tasks:
    model += pulp.lpSum(x[(t, p)] for p in participants) == 1


for p in participants:
    model += pulp.lpSum(
        effort[t] * x[(t, p)] for t in tasks
    ) <= MAX_LOAD



model.solve()


print("Статус:", pulp.LpStatus[model.status])

for p in participants:
    assigned = [t for t in tasks if x[(t, p)].value() == 1]
    load = sum(effort[t] for t in assigned)
    print(f"{p}: {assigned}, нагрузка = {load:.1f} ч")


Статус: Optimal
U1: ['T3.6', 'T3.7', 'T3.9', 'T3.10'], нагрузка = 10.2 ч
U2: ['T3.2', 'T3.3', 'T3.4', 'T3.5'], нагрузка = 10.2 ч
U3: ['T3.1', 'T3.8', 'T3.11'], нагрузка = 11.2 ч
